In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

C:\Users\tzeng1\repos\LIDC_GDRO


In [18]:
# import os
# import torch
# import pandas as pd
# import numpy as np
from dataloaders import InfiniteDataLoader
from datasets import NoduleDataset
from models import TransferModel
from loss import ERMLoss#, GDROLossAlt, GDROLoss
from train import train, test
# from torch.optim.lr_scheduler import ReduceLROnPlateau

# import torchvision
import torch
from image_data_utils import getImages

In [4]:
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Using cpu")
    DEVICE = torch.device("cpu")

Using cpu


#To derive an upper bound on performance, we first do clustering on entire dataset

In [5]:
nodule_id, img_data = getImages(split=False, sublabels=False, binary=False, device=DEVICE)

In [13]:
dataset = NoduleDataset(*img_data)
loader = InfiniteDataLoader(dataset, batch_size=128)

In [16]:
model = TransferModel(freeze=False, binary=False, device=DEVICE)
loss_fn = ERMLoss(model,torch.nn.CrossEntropyLoss(),{})
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.005)

epochs = 15
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train(loader, model, loss_fn, optimizer, verbose=True)
